### Building a RAG System with LangChain and ChromaDB. 
-> RAG is a powerful technique that combines the capabilities of LLMs with external retrieval.

Langchain : framework for building and developing apps powered by language models  
Chroma DB : opensource vector DB for storing and retrieving embeddings. 


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# Langchain imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document


## vector store imports
from langchain_community.vectorstores import Chroma

# utility imports
from typing import List
import numpy as np


In [5]:
# RAG ARCHITECTURE OVERVIEW

print("""
    Retrieval-Augmented Generation (RAG) Architecture Overview:
      1.Document Loading: Load documents from various sources (e.g., text files, PDFs, web pages).
      2.Text Splitting: Split documents into smaller chunks for better processing.
      3.Embedding Generation: Convert text chunks into vector embeddings using pre-trained models.
      4. vector Store Creation: Store embeddings in a vector database for efficient retrieval.
      5.Query Processing: Convert user queries into embeddings.
      6.similarity Search: Retrieve relevant document chunks based on query embeddings.
      7. context augmentation: Combine retrieved documents with the original query.
      8. response Generation: Generate responses using a language model based on the augmented context.

    Benefits of RAG:
        - Enhanced Accuracy: Provides more accurate and contextually relevant responses.
        - reduced Hallucination: Minimizes the chances of generating incorrect information.
        - Up-to-date Information: Allows integration of the latest data without retraining the model.
        - works with domain-specific knowledge: Tailors responses to specific fields or industries.

""")


    Retrieval-Augmented Generation (RAG) Architecture Overview:
      1.Document Loading: Load documents from various sources (e.g., text files, PDFs, web pages).
      2.Text Splitting: Split documents into smaller chunks for better processing.
      3.Embedding Generation: Convert text chunks into vector embeddings using pre-trained models.
      4. vector Store Creation: Store embeddings in a vector database for efficient retrieval.
      5.Query Processing: Convert user queries into embeddings.
      6.similarity Search: Retrieve relevant document chunks based on query embeddings.
      7. context augmentation: Combine retrieved documents with the original query.
      8. response Generation: Generate responses using a language model based on the augmented context.

    Benefits of RAG:
        - Enhanced Accuracy: Provides more accurate and contextually relevant responses.
        - reduced Hallucination: Minimizes the chances of generating incorrect information.
        - Up-to-

In [6]:
## create sample documents
sample_docs = [
    """
    Machine Learning Fundamentals
    
    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There are three main 
    types of machine learning: supervised learning, unsupervised learning, and reinforcement 
    learning. Supervised learning uses labeled data to train models, while unsupervised 
    learning finds patterns in unlabeled data. Reinforcement learning learns through 
    interaction with an environment using rewards and penalties.
    """,
    
    """
    Deep Learning and Neural Networks
    
    Deep learning is a subset of machine learning based on artificial neural networks. 
    These networks are inspired by the human brain and consist of layers of interconnected 
    nodes. Deep learning has revolutionized fields like computer vision, natural language 
    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly 
    effective for image processing, while Recurrent Neural Networks (RNNs) and Transformers 
    excel at sequential data processing.
    """,
    
    """
    Natural Language Processing (NLP)
    
    NLP is a field of AI that focuses on the interaction between computers and human language. 
    Key tasks in NLP include text classification, named entity recognition, sentiment analysis, 
    machine translation, and question answering. Modern NLP heavily relies on transformer 
    architectures like BERT, GPT, and T5. These models use attention mechanisms to understand 
    context and relationships between words in text.
    """
]

sample_docs


['\n    Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through \n    interaction with an environment using rewards and penalties.\n    ',
 '\n    Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision, natural language \n    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly \n    effective f

In [7]:
## store the smaple documents in text files

import tempfile # to create a temporary directory
temp_dir=tempfile.mkdtemp() # create a temporary directory

for i,doc in enumerate(sample_docs): # iterate over sample documents
    with open(f"{temp_dir}/doc_{i}.txt","w") as f: # create and open a text file
        f.write(doc) # write the document content to the file

print(f"Sample document create in : {temp_dir}")

Sample document create in : /var/folders/0x/l0pl9q9552d_t68n7_dctx240000gn/T/tmprcro__xz


In [8]:




for i,doc in enumerate(sample_docs):
    with open(f"doc_{i}.txt","w") as f: # create and open a text file
        f.write(doc) # write the document content to the file



### 2. DOCUMENT LOADING

In [9]:
## Document Loading

from langchain_community.document_loaders import DirectoryLoader

# Load docs from the directory

loader = DirectoryLoader(
    "data", # directory path
    glob="*.txt", # load all text files
    loader_cls=TextLoader,
    loader_kwargs={"encoding":"utf-8"}
)

documents = loader.load() # list of Document objects

print(f" Loaded {len(documents)} documents.")
print("\n First document preview : ")
print(documents[0].page_content[:200])  # print first 200 characters of the first document

 Loaded 3 documents.

 First document preview : 

    Natural Language Processing (NLP)

    NLP is a field of AI that focuses on the interaction between computers and human language. 
    Key tasks in NLP include text classification, named entity r


### 3. DOCUMENT SPLITTING

In [10]:
# intilalize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, # size of each chunk
    chunk_overlap=50, # overlap between chunks
    separators=[" ", ""], # preferred separators
    length_function=len # function to calculate length
)

chunks = text_splitter.split_documents(documents)  # split documents into chunks and store in chunks list

print(f" Total Chunks created : {len(chunks)}")
print("\n First chunk preview : ")
print(f"content: {chunks[0].page_content[:150]}...")  # print first 150 characters of the first chunk
print(f"metadata: {chunks[0].metadata}")  # print metadata of the first chunk


 Total Chunks created : 5

 First chunk preview : 
content: Natural Language Processing (NLP)

    NLP is a field of AI that focuses on the interaction between computers and human language. 
    Key tasks in NL...
metadata: {'source': 'data/doc_2.txt'}


### 4. EMBEDDING MODELS

#### Initialise the ChromaDB vectore store and store the vectors

In [11]:
## create a chromadb vector store

persist_directory = "./chroma_db"  # directory to store the vector database

## intializr chromadb with HuggingFace embeddings

vector_store = Chroma.from_documents(
    documents=chunks, # list of document chunks
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"), # embedding model
    persist_directory=persist_directory, # directory to store the vector database
    collection_name="rag_docs" # name of the collection
)

In [12]:
print(f"vector store created with {vector_store._collection.count()} vectors.")
print(f"persisted at : {persist_directory}")

vector store created with 15 vectors.
persisted at : ./chroma_db


### Test similarity search

In [13]:
query = "What are the main types of machine learning?"

similar_docs = vector_store.similarity_search(
    query=query, # user query
    k=3 # number of similar documents to retrieve
)

similar_docs

[Document(metadata={'source': 'data/doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through'),
 Document(metadata={'source': 'data/doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, whi

In [14]:
## advance similarity search with scores
query = "Explain the concept of deep learning."
similar_docs_with_scores = vector_store.similarity_search_with_score(
    query=query, # user query
    k=3 # number of similar documents to retrieve
)   
similar_docs_with_scores

[(Document(metadata={'source': 'data/doc_1.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision, natural language \n    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly \n    effective for image processing, while Recurrent Neural Networks (RNNs) and Transformers'),
  0.6450324058532715),
 (Document(metadata={'source': 'data/doc_1.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision, natural language \n    processing, and speech recogniti

### Initialize LLM,RAG chain, Prompt Template, Query to the RAG system

In [ ]:
import os

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq

# Now you don't even need to pass the api_key argument; it will find it automatically
llm = ChatGroq(
    model_name="openai/gpt-oss-120b"
)

In [16]:
test_response = llm.invoke(["Explain the concept of reinforcement learning in simple terms."])
test_response

AIMessage(content='**Reinforcement Learning (RL) in a nutshell**\n\nImagine you’re teaching a pet dog to fetch a ball. You don’t write the dog a step‑by‑step manual; instead, you let the dog try things, and you give it treats (rewards) when it does something you like and ignore or gently correct it when it doesn’t. Over time, the dog learns which actions lead to treats and which don’t, and it gets better at fetching the ball quickly and correctly.\n\nReinforcement learning works the same way, just with a computer program (the **agent**) instead of a dog. Here are the key pieces, broken down into everyday language:\n\n| RL Piece | Everyday Analogy | What It Means in RL |\n|----------|------------------|---------------------|\n| **Agent** | The dog (or a kid, a robot, a video‑game character) | The decision‑maker that takes actions. |\n| **Environment** | The backyard, the park, the room where the dog plays | Everything the agent interacts with; it reacts to the agent’s actions. |\n| **St

In [17]:
from langchain.chat_models.base import init_chat_model # to initialize chat model

llm = init_chat_model(
    model="openai/gpt-oss-120b",
    model_provider="groq"
)



In [18]:
response = llm.invoke("what is AI")
response

AIMessage(content='**Artificial Intelligence (AI)** is a branch of computer science that focuses on creating systems capable of performing tasks that normally require human intelligence. These tasks include learning, reasoning, problem‑solving, perception, language understanding, and decision‑making.\n\n### Core ideas\n| Concept | What it means |\n|---------|----------------|\n| **Machine Learning (ML)** | Algorithms that improve their performance automatically through experience (e.g., recognizing images after being shown many examples). |\n| **Deep Learning** | A subset of ML that uses multi‑layered neural networks to model complex patterns, especially effective for speech, vision, and language tasks. |\n| **Natural Language Processing (NLP)** | Techniques for understanding, generating, and translating human language (e.g., chatbots, translators). |\n| **Computer Vision** | Methods for interpreting visual data—identifying objects, faces, scenes, etc. |\n| **Reinforcement Learning** |

### Modern RAG chain


In [20]:
from langchain_core.prompts import ChatPromptTemplate  #to create chat prompt templates
from langchain_classic.chains import create_retrieval_chain # to create retrieval chain 
from langchain_classic.chains.combine_documents import create_stuff_documents_chain # to create stuff documents chain




In [21]:
## convert vector store to retriever
retriever = vector_store.as_retriever(
    search_kwarg = {"k":3} ## retrieve top 3 relevant chunks
)



In [22]:
## create prompt template 

system_prompt = """ You are an assistant for question-answering tasks.
Use the following peices of retrieved context to answer the question.
If you dont know the answer, just say you dont know.
Use three sentences maximum and keep it concise.

Context: {context}"""


prompt = ChatPromptTemplate.from_messages([
    ("system",system_prompt),
    ("human", "{input}")
])

In [23]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' You are an assistant for question-answering tasks.\nUse the following peices of retrieved context to answer the question.\nIf you dont know the answer, just say you dont know.\nUse three sentences maximum and keep it concise.\n\nContext: {context}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [24]:
### create a document chain
doc_chain = create_stuff_documents_chain(llm, prompt)
doc_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' You are an assistant for question-answering tasks.\nUse the following peices of retrieved context to answer the question.\nIf you dont know the answer, just say you dont know.\nUse three sentences maximum and keep it concise.\n\nContext: {context}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image

In [25]:
## create retrieval chain
retrieval_chain = create_retrieval_chain(retriever, doc_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x13f10a120>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' You are an assistant for question-answering tasks.\nUse the following peices of retrieved context to answer the question.\nIf you dont kn

In [26]:
response = retrieval_chain.invoke({"input":"Explain the three main types of machine learning."})

In [27]:
response

{'input': 'Explain the three main types of machine learning.',
 'context': [Document(metadata={'source': 'data/doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through'),
  Document(metadata={'source': 'data/doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcemen

### Create RAG chain alternative - Using LCEL (langchain expression language)

In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [29]:
# create custom prompt

custom_prompt = ChatPromptTemplate.from_template(""" Use the following pieces of context to answer the question.
If you don't have the answer, just say that you don't know.
Provide specific details from the context to support your answer.

Context: {context}

Question: {question}

Answer: """
)

custom_prompt


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template=" Use the following pieces of context to answer the question.\nIf you don't have the answer, just say that you don't know.\nProvide specific details from the context to support your answer.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer: "), additional_kwargs={})])

In [31]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x13f10a120>, search_kwargs={})

In [32]:
## format the output documents for the prompts

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [39]:
# Build the chain using LCEL
ragchain_lcel = (

    {       
        "context": retriever | format_docs, 
        "question": RunnablePassthrough()
     }
    | custom_prompt
    | llm
    | StrOutputParser()
)

ragchain_lcel

{
  context: VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x13f10a120>, search_kwargs={})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template=" Use the following pieces of context to answer the question.\nIf you don't have the answer, just say that you don't know.\nProvide specific details from the context to support your answer.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer: "), additional_kwargs={})])
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reas

In [40]:
response = ragchain_lcel.invoke("what is Deep Learning?")
response

'Deep Learning is a **subset of machine learning** that relies on **artificial neural networks**. These networks are modeled after the human brain and are built from multiple layers of interconnected nodes. By stacking many such layers, deep‑learning models can automatically learn hierarchical representations of data, which has driven breakthroughs in areas like computer vision, natural language processing, and speech recognition. The context explicitly defines deep learning this way:\n\n- “Deep learning is a subset of machine learning based on artificial neural networks.”  \n- “These networks are inspired by the human brain and consist of layers of interconnected nodes.”  \n- “Deep learning has revolutionized fields like computer vision, natural language processing, and speech recognition.”'

In [52]:
# Query Using LCEL approach

def query_rag_lcel(question):
    print(f"Question: {question} ")
    print("-" * 50)

    #pass string directly to LCEL
    answer = ragchain_lcel.invoke(question)
    print("Answer:", answer)

    #Get source documents seperately if needed
    docs = retriever.invoke(question)
    print("\n Source Documents:")
    for i, doc in enumerate(docs):
        print(f"\n --- Source{i+1} ---")
        print(doc.page_content[:200] + "...")

In [53]:
query_rag_lcel("what are the key concepts in reinforcement learning?")

Question: what are the key concepts in reinforcement learning? 
--------------------------------------------------
Answer: The context describes reinforcement learning as being built around three core ideas:

1. **Interaction with an environment** – the learning system (the “agent”) continuously engages with its surroundings, taking actions that affect the state of that environment.  
2. **Rewards and penalties** – the agent receives feedback in the form of positive rewards or negative penalties, which it uses to evaluate how good its actions were.  

These two elements – the ongoing **environment‑agent interaction** and the **reward/penalty feedback** – are presented as the fundamental concepts that define reinforcement learning. (The passage does not mention other concepts such as states, policies, or value functions.)

 Source Documents:

 --- Source1 ---
data. Reinforcement learning learns through 
    interaction with an environment using rewards and penalties....

 --- Source2 --

In [54]:
vector_store

In [55]:
## adding new docs to existing vector store

new_document = """
Reinforcement Learning in Detail

Reinforcement learning (RL) is a type of machine learning where an agent learns to make decisions
by interacting with an environment. The agent receives rewards or penalties based on its actions,
and its goal is to maximize the cumulative reward over time. Key concepts in RL include : states,
actions, rewards, policies, and value functions. Popular RL algorithms include Q-learning, SARSA,
and Deep Q-Networks (DQNs). RL has applications in robotics, game playing, and autonomous systems.
RL has been successfully applied to gameplaying (like AlphGo), robotics, and autonomous systems.
"""


In [56]:
new_document

'\nReinforcement Learning in Detail\n\nReinforcement learning (RL) is a type of machine learning where an agent learns to make decisions\nby interacting with an environment. The agent receives rewards or penalties based on its actions,\nand its goal is to maximize the cumulative reward over time. Key concepts in RL include : states,\nactions, rewards, policies, and value functions. Popular RL algorithms include Q-learning, SARSA,\nand Deep Q-Networks (DQNs). RL has applications in robotics, game playing, and autonomous systems.\nRL has been successfully applied to gameplaying (like AlphGo), robotics, and autonomous systems.\n'

In [57]:
new_doc = Document(
    page_content= new_document,
    metadata={"source": "manual_addition", "topic": "Reinforcement Learning"}
)

In [58]:
new_doc

Document(metadata={'source': 'manual_addition', 'topic': 'Reinforcement Learning'}, page_content='\nReinforcement Learning in Detail\n\nReinforcement learning (RL) is a type of machine learning where an agent learns to make decisions\nby interacting with an environment. The agent receives rewards or penalties based on its actions,\nand its goal is to maximize the cumulative reward over time. Key concepts in RL include : states,\nactions, rewards, policies, and value functions. Popular RL algorithms include Q-learning, SARSA,\nand Deep Q-Networks (DQNs). RL has applications in robotics, game playing, and autonomous systems.\nRL has been successfully applied to gameplaying (like AlphGo), robotics, and autonomous systems.\n')

In [59]:
# split the document
new_chunks = text_splitter.split_documents([new_doc])
new_chunks


[Document(metadata={'source': 'manual_addition', 'topic': 'Reinforcement Learning'}, page_content='Reinforcement Learning in Detail\n\nReinforcement learning (RL) is a type of machine learning where an agent learns to make decisions\nby interacting with an environment. The agent receives rewards or penalties based on its actions,\nand its goal is to maximize the cumulative reward over time. Key concepts in RL include : states,\nactions, rewards, policies, and value functions. Popular RL algorithms include Q-learning, SARSA,\nand Deep Q-Networks (DQNs). RL has applications in robotics, game'),
 Document(metadata={'source': 'manual_addition', 'topic': 'Reinforcement Learning'}, page_content='(DQNs). RL has applications in robotics, game playing, and autonomous systems.\nRL has been successfully applied to gameplaying (like AlphGo), robotics, and autonomous systems.')]

In [60]:
## add new docs to vector store
vector_store.add_documents(new_chunks)

['921ca94e-eb03-4f97-891a-5dccda3c02fa',
 'd669b977-ffcf-48a7-b4f9-7a27e9f3350f']

In [61]:
print(f"added {len(new_chunks)} chunks to vector store")
print(f"Total vectors now are : {vector_store._collection.count()}")

added 2 chunks to vector store
Total vectors now are : 17


In [63]:
new_question = "what are key concepts in Reinforcement Learning?"

result = query_rag_lcel(new_question)

result


Question: what are key concepts in Reinforcement Learning? 
--------------------------------------------------
Answer: The context highlights the core ideas that define reinforcement learning:

- **Interaction with an environment** – the learning agent must act within, and receive feedback from, a surrounding environment.  
- **Rewards and penalties** – the agent’s behavior is guided by receiving positive rewards for desirable actions and penalties (negative rewards) for undesirable ones.  

These two elements—environmental interaction and the reward/penalty feedback loop—are presented in the text as the fundamental concepts of reinforcement learning.

 Source Documents:

 --- Source1 ---
data. Reinforcement learning learns through 
    interaction with an environment using rewards and penalties....

 --- Source2 ---
data. Reinforcement learning learns through 
    interaction with an environment using rewards and penalties....

 --- Source3 ---
data. Reinforcement learning learns thro

### Adv RAG Technique(Conversational Memory)

In [72]:
from langchain_classic.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

In [74]:
## create a prompt that includes chat history
contextualize_q_system_prompt = """Given a chat history and a latest user question,
which might reference context from the chat history, formulate a standalone question
which can be understood without the chat history. Do NOT answer the question.,
just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])


In [75]:
## create history aware retriever

history_aware_retriever = create_history_aware_retriever(
   llm, retriever, contextualize_q_prompt
)

history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x13f10a120>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMes

In [76]:
## create a new document chain with history

qa_system_prompt = """You are a helpful assistant that answers questions.
Use the following pieces of retrieved context to answer the question.
If you don't have the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.

context: {context}"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)


# create a conversational Rag chain
conversational_rag_chain = create_retrieval_chain(
    history_aware_retriever,
    question_answer_chain
)

print("Conversational RAG chain created!!!")

Conversational RAG chain created!!!


In [77]:
chat_history = []

#First Question
result_1 = conversational_rag_chain.invoke({
    "chat_history": chat_history,
    "input": "what is machine learning?"
})

print("Q: what is machine learning?")
print("A: ", result_1["answer"])

Q: what is machine learning?
A:  Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. It uses data—often labeled for supervised learning or unlabeled for unsupervised learning—to automatically discover patterns and make predictions. Reinforcement learning is another approach where models learn by interacting with an environment and receiving feedback.


In [78]:
chat_history.extend([
    HumanMessage(content="what is machine learning?"),
    AIMessage(content=result_1["answer"])
]
)

In [79]:
## follow up question

result_2 = conversational_rag_chain.invoke({
    "chat_history": chat_history,
    "input": "what are its main types?"
})

result_2

{'chat_history': [HumanMessage(content='what is machine learning?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. It uses data—often labeled for supervised learning or unlabeled for unsupervised learning—to automatically discover patterns and make predictions. Reinforcement learning is another approach where models learn by interacting with an environment and receiving feedback.', additional_kwargs={}, response_metadata={})],
 'input': 'what are its main types?',
 'context': [Document(metadata={'source': 'data/doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised lear

In [80]:
result_2['answer']

'The main types of machine learning are supervised learning, unsupervised learning, and reinforcement learning. Supervised learning trains models on labeled data, unsupervised learning discovers patterns in unlabeled data, and reinforcement learning learns by interacting with an environment and receiving feedback.'

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Store chat history
chat_history = []

# Contextualize: Rewrite question using history
def contextualize_question(input_dict):
    if not input_dict["chat_history"]:
        return input_dict["question"]
    
    # Use LLM to rewrite question
    msgs = [
        ("system", "Rewrite the question to be standalone using the chat history."),
        *[(msg.type, msg.content) for msg in input_dict["chat_history"]],
        ("human", input_dict["question"])
    ]
    rewritten = llm.invoke(msgs)
    return rewritten.content

# LCEL Chain with History
conversational_chain = (
    {
        "context": RunnableLambda(contextualize_question) | retriever | format_docs,
        "question": lambda x: x["question"],
        "chat_history": lambda x: x["chat_history"]
    }
    | qa_prompt   # prompt that includes chat_history placeholder
    | llm
    | StrOutputParser()
)

# Usage
response = conversational_chain.invoke({
    "question": "What is deep learning?",
    "chat_history": chat_history
})